In [2]:
import sys
sys.path.insert(0, 'c:\\users\jackg\Documents\python_projects')
from perms_and_combs.perms_and_combs import permutation_no_repetition
from trie import Trie, TrieNode 
from time import time

In [3]:
filename = 'lexicon.txt'
lexicon_file = open(filename, 'r')
lexicon = [line.rstrip() for line in lexicon_file.readlines()]

In [4]:
trie = Trie()
trie.add_words(lexicon[:40000])

In [7]:
letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
# letters = ['A', 'B', 'C']

# Old solution

In [8]:
def get_words(letters, word_length):
    perms = permutation_no_repetition(len(letters), word_length)
    for perm in perms:
        word = ''.join([letters[i] for i in perm])
        if word in trie:
            yield word

In [13]:
words = []
for word_length in range(1, len(letters)+1):
    start = time()
    g = list(get_words(letters, word_length))
    words += g
    end = time()
    print('Time taken for length {} is {}. Found {} words'.format(word_length, end-start, len(g)))

Time taken for length 1 is 0.0. Found 0 words
Time taken for length 2 is 0.0. Found 10 words
Time taken for length 3 is 0.005982875823974609. Found 21 words
Time taken for length 4 is 0.021938562393188477. Found 28 words
Time taken for length 5 is 0.06881499290466309. Found 17 words
Time taken for length 6 is 0.2715587615966797. Found 4 words
Time taken for length 7 is 0.9175450801849365. Found 1 words
Time taken for length 8 is 2.213120222091675. Found 0 words
Time taken for length 9 is 3.420853614807129. Found 0 words


In [14]:
len(words)

81

# New solution

In [15]:
words = trie.find_words(letters)

In [16]:
len(words)

81